In [3]:
# set up imports
import pandas as pd, numpy as np, seaborn as sns
from tabulate import tabulate
import statsmodels
import arch
import matplotlib
matplotlib.use('qt5agg')

# configure plot style
import matplotlib.pyplot as plt
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams["figure.figsize"] = (9.5,4.15)
plt.rcParams['figure.constrained_layout.use'] = False
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100
plt.rcParams['lines.linewidth'] = 0.8
save_plot_to =  r'C:\\Users\joche\OneDrive\03 TUM - TUM-BWL\Semester 8\01 Bachelorarbeit\04 Results\Plots/'

In [4]:
# load data and crop to start_date:end_date
start_date = '2015-08-07'
end_date = '2020-06-26'
idx = pd.date_range(start_date, end_date)
index_name = 'date'
mydateparser = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
mydateparser1 = lambda x: pd.datetime.strptime(str(x), '%Y-%m-%d')

# btc
btc = pd.read_excel('Data/BTC_closing.xlsx', parse_dates=[0], index_col=0, date_parser=mydateparser)[start_date:end_date]
btc.index.name = index_name
btc.columns = ['btc']

# usd_eur
usd_eur = pd.read_excel('Data/DEXUSEU.xls', parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[start_date:end_date]
usd_eur.index.name = index_name
usd_eur.columns = ['usd_eur']
usd_eur = usd_eur.loc[(usd_eur!=0).any(1)]

# tot_btc: only weekly data - missing values interpolated
tot_btc = pd.read_csv('Data/total-bitcoins', index_col=0)[start_date:end_date]
tot_btc.index = pd.DatetimeIndex(tot_btc.index, normalize=True).normalize()
tot_btc.index.name = index_name
tot_btc = tot_btc.reindex(idx, fill_value=None)
tot_btc.interpolate(method='time', inplace=True, limit_direction='both')
tot_btc.columns = ['tot_btc']

# hs_rate: only weekly data - missing values interpolated
hs_rate = pd.read_csv('Data/hash-rate', index_col=0)[start_date:end_date]
hs_rate.index = pd.DatetimeIndex(hs_rate.index, normalize=True).normalize()
hs_rate.index.name = index_name
hs_rate = hs_rate.reindex(idx, fill_value=None)
hs_rate.interpolate(method='time', inplace=True, limit_direction='both')
hs_rate.columns = ['hs_rate']

# eth
eth = pd.read_excel('Data/ETH.xlsx', parse_dates=[0], index_col=0, date_parser=mydateparser, usecols='A,E')[start_date:end_date]
eth.index.name = index_name
eth.columns = ['eth']

# ggl_trends: only weekly data - missing values interpolated
# ggl_trends: weighted average of 15 countries
ggl_trends = pd.read_csv('Data/googletrends.txt', index_col=0)[start_date:end_date]
ggl_trends.index = pd.DatetimeIndex(ggl_trends.index, normalize=True).normalize()
ggl_trends.index.name = index_name
ggl_trends = ggl_trends.reindex(idx, fill_value=None)
ggl_trends.interpolate(method='time', inplace=True, limit_direction='both')
ggl_trends['btc-average'] = np.round(ggl_trends.sum(axis=1)/15)
ggl_trends = ggl_trends[['btc-average']].copy()
ggl_trends.columns = ['ggl_trends']

# wiki_views: sum of 99 countries
wiki_views = pd.read_excel('Data/wikipedia.xlsx', parse_dates=[0], index_col=0, date_parser=mydateparser1)[start_date:end_date]
wiki_views.index = pd.DatetimeIndex(wiki_views.index, normalize=True).normalize()
wiki_views.index.name = index_name
wiki_views['wiki-total'] = np.round(wiki_views.sum(axis=1))
wiki_views = wiki_views[['wiki-total']].copy()
wiki_views.columns = ['wiki_views']

# wti_oil
oil_wti = pd.read_excel('Data/DCOILWTICO.xls', parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[start_date:end_date]
oil_wti.index = pd.DatetimeIndex(oil_wti.index, normalize=True).normalize()
oil_wti.index.name = index_name
oil_wti.columns = ['oil_wti']
oil_wti = oil_wti.loc[(oil_wti!=0).any(1)]

# gold
gold = pd.read_excel('Data/GOLDAMGBD228NLBM10AM.xls', parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[start_date:end_date]
gold.index = pd.DatetimeIndex(gold.index, normalize=True).normalize()
gold.index.name = index_name
gold.columns = ['gold']
gold = gold.loc[(gold!=0).any(1)]

# sp500
sp500 = pd.read_excel('Data/SP500.xls', parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[start_date:end_date]
sp500.index = pd.DatetimeIndex(sp500.index, normalize=True).normalize()
sp500.index.name = index_name
sp500.columns = ['sp500']
sp500 = sp500.loc[(sp500!=0).any(1)]

# sse 
sse = pd.read_excel('Data/SSEcomposite.xlsx', parse_dates=[0], index_col=0, header=0, date_parser=mydateparser)[start_date:end_date]
sse.index = pd.DatetimeIndex(sse.index, normalize=True).normalize()
sse.index.name = index_name
sse = sse[['Zuletzt']].copy()
sse.columns = ['sse']

# ffd_rate
ffd_rate = pd.read_excel('Data/DFF.xls', parse_dates=[0], index_col=0, skiprows=10, date_parser=mydateparser)[start_date:end_date]
ffd_rate.index = pd.DatetimeIndex(ffd_rate.index, normalize=True).normalize()
ffd_rate.index.name = index_name
ffd_rate.columns = ['ffd_rate']

In [5]:
# merge data to one df (inner join)
from functools import reduce
temp = [btc,tot_btc,hs_rate,eth,ggl_trends,wiki_views,usd_eur,oil_wti,
              gold,sp500,sse,ffd_rate]
df = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), temp)
df

In [6]:
# define plotting function
import matplotlib.dates as mdates
def plot_df(df):
    fig_i, axs = plt.subplots(4,3, figsize=(9.5,4.15))
    for i, ax in enumerate(axs.flatten()):
        data = df[df.columns[i]]
        ax.plot(data, color='black')
        ax.set_title(df.columns[i])
        ax.xaxis.set_ticks_position('none')
        ax.yaxis.set_ticks_position('none')
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
        ax.xaxis.set_minor_formatter(mdates.DateFormatter("%Y-%m"))
        #ax.spines['top'].set_alpha(0)
        ax.tick_params(labelsize=8)
    plt.tight_layout()
    return fig_i

In [111]:
# plot raw time series
fig_1 = plot_df(df)

In [7]:
# take log of time series and relabel columns
df_log = np.log(df)
new_cols = list()
for i in df_log.columns:
    new_cols.append(i+'_log')
df_log.columns = new_cols

# fill na value of negative oil price on 2020-04-20 with 0
df_log.fillna(value=0, inplace=True)

In [10]:
# plot log time series
fig_2 = plot_df(df_log)

In [113]:
# plot correlation matrix as latex table
corr = df.corr().round(3)
print(tabulate(corr, headers=corr.columns, showindex=True, tablefmt="latex"))

In [114]:
# plot correlation matrix as heatmap
corr = df.corr()
# more html colors here: https://www.w3schools.com/colors/colors_names.asp
# pal = sns.light_palette('lightgrey', as_cmap=True)
ax = sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot=True, 
            annot_kws={'size':7}, vmin=-1, center=0, vmax=1, cmap="YlGnBu")
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
fig_3 = plt.gcf()
fig_3.set_size_inches(9.5,4.15)
plt.tick_params(axis='both', which='major', labelsize=7.5, labelbottom = False, bottom=False, top = False, labeltop=True)
plt.xticks(fontsize=7.5, rotation=1)
plt.yticks(fontsize=7.5)

In [8]:
# split df in df_train and df_test set
train_size = int(len(df) * 0.8)
df_train, df_test = df[0:train_size], df[train_size:]
print('df:\t\t',len(df))
print('df_train:\t',len(df_train))
print('df_test:\t',len(df_test))
print('Sum train+test:\t', len(df_train)+len(df_test))

In [9]:
# take log of df_train: df_train_log
df_train_log = np.log(df_train)

# relabel columns
new_cols = list()
for i in df_train_log.columns:
    new_cols.append(i+'_log')
df_train_log.columns = new_cols

df_train_log

In [117]:
# plot df_train_log
fig_4 = plot_df(df_train_log)

In [118]:
# adf- and pp-test of a df
from arch.unitroot import ADF, PhillipsPerron
def stationarity_tests(df, latex):
    for col in df:
        adf = ADF(df[col])
        pp = PhillipsPerron(df[col])
        
        if latex is False:
            # write summary as plain text to std.out
            print('Timeseries:\t',col,'\n',
                  adf.summary(),'\n\n',pp.summary(),'\n\n\n')
        else:
            # write summary as latex to file
            with open(save_plot_to + 'Stationarity_Tests_LaTeX.txt', 'a') as myfile:
                myfile.write('Timeseries:\t'+col+'\n'
                         +adf.summary().as_latex()+'\n\n'
                         +pp.summary().as_latex()+'\n\n\n')

In [119]:
# adf- and pp-test of df_train_log
stationarity_tests(df_train_log, latex=False)

In [12]:
# first differences of df_train_log: df_train_log_diff
df_train_log_diff = df_train_log.diff()

# relabel columns
new_cols = list()
for i in df_train_log_diff.columns:
    new_cols.append(i+'_diff')
df_train_log_diff.columns = new_cols

df_train_log_diff

In [121]:
# plot df_train_log_diff
fig_5 = plot_df(df_train_log_diff[1:])

In [122]:
# adf- and pp-test of df_train_log_diff
stationarity_tests(df_train_log_diff[1:], latex=False)

In [123]:
# granger causality test on a df
# code taken from: https://stackoverflow.com/questions/58005681/is-it-possible-to-run-a-vector-autoregression-analysis-on-a-large-gdp-data-with
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=15
test = 'ssr-chi2test'
def grangers_causality_matrix(X_train, variables, test = 'ssr_chi2test', verbose=False):
    dataset = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in dataset.columns:
        for r in dataset.index:
            test_result = grangercausalitytests(X_train[[r,c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            dataset.loc[r,c] = min_p_value
    dataset.columns = [var + '_x' for var in variables]
    dataset.index = [var + '_y' for var in variables]
    return dataset

In [124]:
# perform granger causality test each on each based on df_train_log_diff
granger_causality_matrix = grangers_causality_matrix(df_train_log_diff[1:], variables = df_train_log_diff.columns)
granger_causality_matrix.round(4)

In [125]:
# plot granger causality matrix as latex table
print(tabulate(granger_causality_matrix, headers=granger_causality_matrix.columns, showindex=True, tablefmt="latex"))

In [89]:
# plot granger causality matrix as heatmap
ax = sns.heatmap(granger_causality_matrix, xticklabels=granger_causality_matrix.columns.values, yticklabels=granger_causality_matrix.index.values,
                 annot=True, annot_kws={'size':7}, vmin=0, vmax=1, cmap="YlGnBu_r")
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
fig_6 = plt.gcf()
fig_6.set_size_inches(9.5,4.15)
plt.tick_params(axis='both', which='major', labelsize=7, labelbottom = False, bottom=False, top = False, labeltop=True)
plt.xticks(fontsize=7, rotation=45)
plt.yticks(fontsize=7)

In [13]:
# Initiiere VAR-Modell
from statsmodels.tsa.vector_ar.var_model import VAR
# Order selection
var_model = VAR(endog=df_train_log_diff[1:])
var_order_res = var_model.select_order(15).summary()
res_as_html = var_order_res.as_html()
df_var_order = pd.read_html(res_as_html, header=0, index_col=0)[0]
df_var_order

In [131]:
# plot df_var_order as latex table
print(tabulate(df_var_order, headers=df_var_order.columns, showindex=True, tablefmt="latex"))

In [28]:
# fit VAR model
var_model = VAR(endog=df_train_log_diff[1:])
var_fitted = var_model.fit(maxlags = 5)
var_fitted.summary()

C:\Users\joche\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:583: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 05, Sep, 2020
Time:                     16:21:09
--------------------------------------------------------------------
No. of Equations:         12.0000    BIC:                   -91.7375
Nobs:                     883.000    HQIC:                  -94.1867
Log likelihood:           27949.7    FPE:                2.74113e-42
AIC:                     -95.7028    Det(Omega_mle):     1.22969e-42
--------------------------------------------------------------------
Results for equation btc_log_diff
                            coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------------
const                          0.001063         0.003756            0.283           0.777
L1.btc_log_diff                0.011233         0.039179            0.287           0.774
L1.tot_btc_log_di

In [148]:
# transform var_fitted into data frame
# code taken from: https://stackoverflow.com/questions/51734180/converting-statsmodels-summary-object-to-pandas-dataframe
def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher
                                })
    #Reordering
    results_df = results_df[["coeff","pvals","conf_lower","conf_higher"]]
    return results_df

In [23]:
# read var coefficients for btc from .txt file
var_btc_coefficients = pd.read_csv(save_plot_to+'VAR_coefficients.txt', delimiter='\s+', header=0)
var_btc_coefficients.set_index(var_btc_coefficients.columns[0], inplace=True)

# plot var_btc_coefficients as latex table
print(tabulate(var_btc_coefficients.round(4), headers=var_btc_coefficients.columns, showindex=True, tablefmt="latex"))

\begin{tabular}{lrrrr}
\hline
                        &   coefficient &   std.error &   t-stat &   prob \\
\hline
 const                  &        0.0011 &      0.0038 &    0.283 &  0.777 \\
 L1.btc\_log\_diff        &        0.0112 &      0.0392 &    0.287 &  0.774 \\
 L1.tot\_btc\_log\_diff    &      -10.5357 &      8.3328 &   -1.264 &  0.206 \\
 L1.hs\_rate\_log\_diff    &        0.1549 &      0.093  &    1.665 &  0.096 \\
 L1.eth\_log\_diff        &       -0.0105 &      0.023  &   -0.455 &  0.649 \\
 L1.ggl\_trends\_log\_diff &        0.0307 &      0.0226 &    1.362 &  0.173 \\
 L1.wiki\_views\_log\_diff &       -0.0109 &      0.0097 &   -1.127 &  0.26  \\
 L1.usd\_eur\_log\_diff    &       -0.2916 &      0.3442 &   -0.847 &  0.397 \\
 L1.oil\_wti\_log\_diff    &       -0.0571 &      0.0751 &   -0.761 &  0.447 \\
 L1.gold\_log\_diff       &       -0.0647 &      0.2283 &   -0.283 &  0.777 \\
 L1.sp500\_log\_diff      &        0.3629 &      0.2069 &    1.754 &  0.079 \\
 L1.sse\_log\

In [68]:
# forecast
# get lag order
lag_order = var_fitted.k_ar

# input data
input_data = df_train_log_diff.dropna().values[-lag_order:]

# forecasting
pred = var_fitted.forecast(y=input_data, steps=len(df_test))
pred = pd.DataFrame(pred, index=df_test.index, columns=df_test.columns + '_pred')
pred

,btc_pred,tot_btc_pred,hs_rate_pred,eth_pred,ggl_trends_pred,wiki_views_pred,usd_eur_pred,oil_wti_pred,gold_pred,sp500_pred,sse_pred,ffd_rate_pred
date,,,,,,,,,,,,
2019-07-02,0.002316,0.000160,-0.004694,0.005866,-0.030162,-0.014930,0.002244,0.018227,-0.000825,0.000100,0.006029,0.023496
2019-07-03,-0.010462,0.000104,-0.002276,0.013481,-0.008773,0.011432,-0.000534,-0.008245,0.001935,0.002868,0.000919,-0.022657
2019-07-08,-0.009699,0.000191,-0.000052,-0.026731,-0.033723,-0.049527,0.000266,-0.002345,-0.000444,-0.005059,-0.003555,0.035934
2019-07-09,-0.006557,0.000237,-0.002195,-0.020945,-0.025351,-0.040259,-0.000566,0.002509,0.000400,0.002426,0.002326,0.000584
2019-07-10,0.001899,0.000242,0.000157,0.002297,0.012449,-0.019038,0.000124,-0.001124,0.000010,-0.003027,-0.003045,0.006693
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-18,0.004098,0.000231,0.005663,0.005752,0.002301,0.000493,0.000021,0.000377,0.000241,0.000382,-0.000299,0.003159
2020-06-19,0.004098,0.000231,0.005663,0.005752,0.002301,0.000493,0.000021,0.000377,0.000241,0.000382,-0.000299,0.003159
2020-06-22,0.004098,0.000231,0.005663,0.005752,0.002301,0.000493,0.000021,0.000377,0.000241,0.000382,-0.000299,0.003159


In [88]:
# rolling forecast ?

date
2019-07-02    0.002316
2019-07-03   -0.010462
2019-07-08   -0.009699
2019-07-09   -0.006557
2019-07-10    0.001899
                ...   
2020-06-18    0.004098
2020-06-19    0.004098
2020-06-22    0.004098
2020-06-23    0.004098
2020-06-24    0.004098
Name: btc_pred, Length: 223, dtype: float64

In [84]:
# inverting transformation: df_train_log_diff -> df
def invert_transformation(df_train, pred_df):
    forecast = pred.copy()
    columns = df_train.columns
    for col in columns:
        forecast[str(col)+'_pred'] = df_train[col].iloc[-1] + forecast[str(col)+'_pred'].cumsum()
    return forecast

output = invert_transformation(df_train[1:], pred)
output

#output_original = np.exp(output)

,btc_pred,tot_btc_pred,hs_rate_pred,eth_pred,ggl_trends_pred,wiki_views_pred,usd_eur_pred,oil_wti_pred,gold_pred,sp500_pred,sse_pred,ffd_rate_pred
date,,,,,,,,,,,,
2019-07-02,10583.132316,1.779160e+07,5.985333e+07,293.645866,16.969838,28985.985070,1.132944,58.928227,1390.049175,2964.330100,3044.906029,2.413496
2019-07-03,10583.121854,1.779160e+07,5.985333e+07,293.659347,16.961065,28985.996502,1.132410,58.919982,1390.051110,2964.332967,3044.906948,2.390840
2019-07-08,10583.112155,1.779160e+07,5.985333e+07,293.632616,16.927342,28985.946975,1.132676,58.917637,1390.050665,2964.327908,3044.903393,2.426773
2019-07-09,10583.105598,1.779160e+07,5.985333e+07,293.611671,16.901991,28985.906716,1.132109,58.920146,1390.051065,2964.330335,3044.905719,2.427358
2019-07-10,10583.107497,1.779160e+07,5.985333e+07,293.613967,16.914440,28985.887677,1.132233,58.919022,1390.051075,2964.327308,3044.902674,2.434051
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-18,10583.964476,1.779160e+07,5.985334e+07,294.808491,17.360364,28985.954647,1.136416,59.000144,1390.101316,2964.408720,3044.835871,3.101145
2020-06-19,10583.968573,1.779160e+07,5.985334e+07,294.814243,17.362665,28985.955140,1.136437,59.000521,1390.101558,2964.409102,3044.835572,3.104304
2020-06-22,10583.972671,1.779160e+07,5.985334e+07,294.819995,17.364966,28985.955632,1.136457,59.000898,1390.101799,2964.409484,3044.835274,3.107463


In [85]:
# combine btc predicted vs. actuals
combine = pd.concat([output['btc_pred'], df_test['btc']], axis=1)
combine

,btc_pred,btc
date,,
2019-07-02,10583.132316,10801.68
2019-07-03,10583.121854,11961.27
2019-07-08,10583.112155,12285.96
2019-07-09,10583.105598,12573.81
2019-07-10,10583.107497,12156.51
...,...,...
2020-06-18,10583.964476,9411.84
2020-06-19,10583.968573,9288.02
2020-06-22,10583.972671,9648.72


In [86]:
# plot
combine.plot()
